In [4]:
import pandas as pd
import datetime 
import numpy as np


In [5]:
customers = pd.read_csv('olist_customers_dataset.csv')
orders = pd.read_csv('olist_orders_dataset.csv', parse_dates=True )
goods = pd.read_csv('olist_order_items_dataset.csv')

In [6]:
# Переводим колонки в формат даты
orders[['order_purchase_timestamp', 
        'order_approved_at', 
        'order_delivered_carrier_date',
        'order_delivered_customer_date',
        'order_estimated_delivery_date']] = orders[['order_purchase_timestamp', 
                                                    'order_approved_at', 
                                                    'order_delivered_carrier_date',
                                                    'order_delivered_customer_date',
                                                    'order_estimated_delivery_date']].apply (pd.to_datetime)
goods['shipping_limit_date'] = pd.to_datetime(goods['shipping_limit_date'])

In [ ]:
#Сколько у нас пользователей, которые совершили покупку только один раз?

In [7]:
# Объединяем данные по заказам и данные по клиентам
# Находим количество покупок каждого клиента и выбираем тех, у кого количество покупок равно 1.
# Покупкой считается зазаказ со статусами "Отгружен" или "Доставлен", а также те, по которым была оплата
new_df = customers.merge(orders, on='customer_id')
new_df.query('order_status=="shipped" or order_status=="delivered" or order_approved_at.notna()')\
    .groupby('customer_unique_id', as_index=False)\
    .agg({'customer_unique_id':'value_counts'})\
    .rename(columns = {'customer_unique_id':'total_sales'})\
    .query('total_sales == 1')\
    .shape[0]

93061

In [ ]:
# Сколько заказов в месяц в среднем не доставляется по разным причинам (вывести детализацию по причинам)?

In [28]:
new_df = orders.merge(goods, on='order_id')
# Пишем функцию которая проверяет каждую строку датафрейма и возвращает значение в завимости от выполненных условий
# Недоставленным считается товар, у которого дата доставки превышает обещанную дату доставки
# 1 причина - Логистическая. Когда дата передачи товара в логистическую службу превысли максимальную дату передачи товара продавцом
#  в лог. службу
# 2 причина - долгое оформление заказа. Когда от создания заказа до его оплаты прошло больше 2 недель.
# 3 причина - другие. 
def classify_order_problems(row):
    if row['order_delivered_customer_date'] > row['order_estimated_delivery_date']:
        if (row['order_delivered_carrier_date'] > row['shipping_limit_date']): 
            return 'logistic_problem'
        elif pd.notnull(row['order_approved_at']) and pd.notnull(row['order_purchase_timestamp']) and (row['order_approved_at'] - row['order_purchase_timestamp']) > np.timedelta64(14, 'D'):
            return 'long_order_proccesing'
        else:
            return 'other_problems'
    
# Применяем функцию к датафрейму        
new_df['reason_order_delay'] = new_df.apply(classify_order_problems, axis=1)

# Находим период в месяцах за который проводится подсчет
min_date = new_df[['order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date']].min().min()
max_date = new_df[['order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date']].max().max()
period = max_date - min_date
months = period.days/30

# Находим общее количество недоставленных товаров в зависимости от причины и делим на количество месяцев в рассматриваемом периоде
# для нахождения среднего количества недоставок в месяц
delivery_problems = new_df.groupby('reason_order_delay', as_index=False)\
                    .agg({'order_id':'count'})\
                    .rename(columns={'order_id':'count_delivery_problems'}) 
delivery_problems['avg_month_delivery_problems'] = delivery_problems['count_delivery_problems'] / months
delivery_problems

,reason_order_delay,count_delivery_problems,avg_month_delivery_problems
0,logistic_problem,2474,96.514954
1,long_order_proccesing,7,0.273082
2,other_problems,6234,243.198960


In [ ]:
# По каждому товару определить, в какой день недели товар чаще всего покупается

In [221]:
#Объединяем таблицу заказов и товаров
goods_in_orders = orders.merge(goods, on='order_id', how='inner')

# Добавляем колонку с днем недели, когда осуществилась продажа товара. Днем продажи считается день, когда прошла оплата за товар
goods_in_orders ['day_of_week'] = goods_in_orders.order_approved_at.dt.dayofweek
days = {0: 'Понедельник', 1: 'Вторник', 2: 'Среда', 3: 'Четверг', 4: 'Пятница', 5: 'Суббота', 6: 'Воскресенье'}
goods_in_orders['day_of_week'] = goods_in_orders['day_of_week'].map(days)

# Делаем группировку по каждому id товара и дню его продажи
df_grouped = goods_in_orders.groupby(['product_id', 'day_of_week']).size().reset_index(name='total_count')


# Находим день недели с максимальным количеством продаж для каждого товара
df_max_sales = df_grouped.loc[df_grouped.groupby(['product_id'])['total_count'].idxmax()].sort_values('total_count', ascending=False)
df_max_sales

,product_id,day_of_week,total_count
41078,aca2eb7d00ea1a7b8ebd4e68314663af,Вторник,119
20148,53b36df67ebb7c41585e8d54d6772e08,Вторник,105
15929,422879e10f46682990de24d770e7f83d,Вторник,89
36570,99a4788cb24856965c36a24e339b6058,Вторник,82
13062,368c6c730842d78016ad823897a372db,Четверг,80
...,...,...,...
23651,62874f62bbdeec35f7cd827b1716c47e,Среда,1
23649,628677036f276f6c0792cded8e9211ee,Среда,1
23647,6281a79f478d260d81da138f7e03c0bb,Пятница,1
23646,627dce1ab24323cb7ac147cad5ac0b8c,Пятница,1


In [1]:
# Сколько у каждого из пользователей в среднем покупок в неделю (по месяцам)? 
# Не стоит забывать, что внутри месяца может быть не целое количество недель. 
# Например, в ноябре 2021 года 4,28 недели. И внутри метрики это нужно учесть. (8 баллов) 

SyntaxError: invalid syntax (<ipython-input-1-5463edfab149>, line 1)

In [4]:
# Объединями данные о заказах и данным о покупателях. 
# Добавляем новый столбец с месяцем продажи. Месяцем продажи считается месяц, в котором прошла оплата за товар.
df = orders.merge(customers, on ='customer_id', how='inner')
df['month'] = df['order_approved_at'].dt.month

# Считаем количество заказов для каждого пользователя по месяцам
monthly_orders = df.groupby(['customer_unique_id', 'month'])\
                .agg({'order_id': 'count'})\
                .reset_index()\
                .rename(columns={'order_id':'count_orders'})\
                .sort_values('count_orders', ascending=False)

days_in_month = {
    1: 31,  # Январь
    2: 28,  # Февраль,
    3: 31,  # Март
    4: 30,  # Апрель
    5: 31,  # Май
    6: 30,  # Июнь
    7: 31,  # Июль
    8: 31,  # Август
    9: 30,  # Сентябрь
    10: 31, # Октябрь
    11: 30, # Ноябрь
    12: 31  # Декабрь
}
# Добавляем новый столбец с количеством недель в месяце
monthly_orders['weeks_in_month'] = monthly_orders['month'].map(days_in_month) / 7

# Считаем среднее количество заказов в неделю для каждого пользователя
monthly_orders['avg_sales_in_week'] = monthly_orders['count_orders'] / monthly_orders['weeks_in_month']

# Выводим 100 покупателей с наибольшим средним количеством заказов в неделю
monthly_orders.sort_values(['avg_sales_in_week'], ascending=False)[['customer_unique_id', 'avg_sales_in_week']].head(100)

,customer_unique_id,avg_sales_in_week
7313,12f5d6e1cbf93dafd9dcc19095df0b3d,1.354839
53938,8d50f5eadf50201ccdcedfb9e2ac8455,1.129032
69267,b4e4f24de1e8725b74e4a1f4975116ed,1.000000
23897,3e43e6105506432c953e165fb2acf44c,1.000000
62081,a239b8e2fbce33780f1f1912e2ee5275,1.000000
...,...,...
79887,d0be859f85cda9d896ab14e4ffba955d,0.500000
42507,6ef4101366e4340921398ba91fff1426,0.500000
10447,1b270bd97f6e1d9af2d5d06f0d4a80e8,0.500000
62971,a48616b0798c3f1d5cae097c18180be8,0.500000


In [ ]:
# В период с января по декабрь выявите когорту с самым высоким retention на 3-й месяц. 

In [225]:
# Обьединяем таблицу заказов с таблицей покупателей
df = orders.merge(customers, on ='customer_id', how='inner')

# Преобразуем столбец с датой создания заказа в формат datetime
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])

# Добавляем колонку с  месяцем первого заказа для каждого пользователя
df['first_order'] = df.groupby('customer_unique_id')['order_purchase_timestamp'].transform('min').dt.to_period('M')

# Добавляем колонку с количеством месяцев, прошедших с момента первого заказа
df['delta'] = (df['order_purchase_timestamp'].dt.to_period('M') - df['first_order'])\
                .apply(lambda x: x.n)

# Группируем данные по когорте и месяцам с момента первого заказа
kogortny_analysis = df.groupby(['first_order', 'delta']) \
                      .agg({'customer_unique_id': 'nunique'}) \
                      .rename(columns={'customer_unique_id': 'count_active_users'}) \
                      .sort_values('first_order', ascending=False)\
# Сбрасываем индекс         
kogortny_analysis.reset_index(inplace=True)

# Создаем сводную таблицу 
pivot_tab = kogortny_analysis\
    .query('delta<13 and first_order>"2016-12" and first_order<"2018-01"')\
    .pivot(index="first_order", columns="delta", values="count_active_users")
pivot_tab

# Применяем к сводной таблице визуальное оформление 
ur_style = (pivot_tab
            .style
            .set_caption('User retention by cohort')  # добавляем подпись
            .background_gradient(cmap='viridis')  # раскрашиваем ячейки по столбцам
            .highlight_null('white'))  #  NaN заменяем на пустоту
ur_style

# САМЫЙ ВЫСОКИЙ RETENTION НА 3 МЕСЯЦ В КОГОРТЕ, ГДЕ ПЕРВЫЙ ЗАКАЗ БЫЛ СДЕЛАН 2017-11

delta,0,1,2,3,4,5,6,7,8,9,10,11,12
first_order,,,,,,,,,,,,,
2017-01,764.000000,3.000000,2.000000,1.000000,3.000000,1.000000,4.000000,1.000000,1.000000,nan,3.000000,1.000000,6.000000
2017-02,1752.000000,4.000000,5.000000,2.000000,7.000000,2.000000,4.000000,3.000000,3.000000,4.000000,2.000000,5.000000,3.000000
2017-03,2636.000000,13.000000,10.000000,10.000000,9.000000,4.000000,4.000000,8.000000,9.000000,2.000000,10.000000,4.000000,6.000000
2017-04,2352.000000,14.000000,5.000000,4.000000,8.000000,6.000000,8.000000,7.000000,7.000000,4.000000,6.000000,2.000000,2.000000
2017-05,3596.000000,18.000000,18.000000,14.000000,11.000000,12.000000,15.000000,6.000000,9.000000,11.000000,9.000000,12.000000,9.000000
2017-06,3139.000000,15.000000,11.000000,13.000000,8.000000,12.000000,12.000000,7.000000,4.000000,7.000000,10.000000,11.000000,5.000000
2017-07,3894.000000,20.000000,14.000000,10.000000,11.000000,8.000000,12.000000,4.000000,7.000000,10.000000,9.000000,12.000000,5.000000
2017-08,4184.000000,29.000000,14.000000,11.000000,15.000000,22.000000,12.000000,11.000000,6.000000,6.000000,10.000000,8.000000,4.000000
2017-09,4130.000000,28.000000,22.000000,12.000000,19.000000,9.000000,9.000000,10.000000,12.000000,7.000000,11.000000,3.000000,nan


In [ ]:
# RFM - сегментация.

In [226]:
# Объединяем данные по покупателям, заказам и товарам в заказах
df = customers.merge(orders, on ='customer_id').merge(goods, on='order_id')

# Убираем из столбцов с датами время
columns_to_convert = ['order_purchase_timestamp', 
                      'order_approved_at', 
                      'order_delivered_carrier_date',
                      'order_delivered_customer_date',
                      'order_estimated_delivery_date']

for column in columns_to_convert:
    orders[column] = pd.to_datetime(orders[column]).dt.date
    
# Определяем текущую дату  как дату создания последнего заказа + 1 день.
from datetime import timedelta
period = 365
NOW = df['order_purchase_timestamp'].max() + timedelta(days=1)


In [228]:
# Для каждого покупателя считаем общее количество сделанных за все время заказов, а также общую сумму покупок
# Покупкой считается заказ, по которому есть либо дата оплаты, либо он доставлен или отгружен

new_df = df.query('order_approved_at.notna() or order_status=="shipped" or order_status=="delivered"')\
    .groupby(['customer_unique_id','order_purchase_timestamp'])\
    .agg({'price': 'sum', 'order_id': 'nunique'})\
    .reset_index()\
    .rename(columns={'price':'total_sum', 'order_id':'count_orders'})

In [229]:
# Добавляем колонку с количеством дней, прошедших с момента заказа
new_df['days_since_order'] = new_df['order_purchase_timestamp'].apply(lambda x: (NOW - x).days)

In [230]:
# Находим сумму покупок каждого покупателя за последний год
total_for_last_year = new_df[new_df['order_purchase_timestamp'] >= NOW - timedelta(days=period)]\
            .groupby('customer_unique_id')\
            .agg({'total_sum': 'sum'})\
            .reset_index()\
            .rename(columns={'total_sum':'total_sum_for_last_year'})\
            .sort_values('total_sum_for_last_year')


In [231]:
# Делаем группировку по покупателю и для каждого покупателя выводим дату последней покупки и частоту покупок за последний год
aggr = {
    'days_since_order': lambda x: x.min(),  
    'order_purchase_timestamp': lambda x: len([d for d in x if d >= NOW - timedelta(days=period)])
}

rfm = new_df.groupby('customer_unique_id')\
        .agg(aggr)\
        .reset_index()\
        .rename(columns = {'days_since_order': 'Recency', 
                           'order_purchase_timestamp': 'Frequency' , 
                           'total_sum_for_last_year': 'Monetary'})

In [232]:
# Обьединяем 2 датафрейма и для каждого покупателя добавляем сумму покупок за последний год.
# Для покупателей, которые за последний год покупок не сделали, заполняем пустые значения нулями
rfm = rfm.merge(total_for_last_year, how='left', on='customer_unique_id')
rfm['total_sum_for_last_year'].fillna(0, inplace=True)
rfm = rfm.rename(columns={'total_sum_for_last_year':'Monetary'})

In [233]:
# Определяем кластеры 
quintiles = rfm[['Recency', 'Frequency', 'Monetary']].quantile([.2, .4, .6, .8]).to_dict()

# Количество дней с последней покупки делим на 5 групп на основании квантилей
def r_score(x):
    if x <= quintiles['Recency'][.2]:
        return 5
    elif x <= quintiles['Recency'][.4]:
        return 4
    elif x <= quintiles['Recency'][.6]:
        return 3
    elif x <= quintiles['Recency'][.8]:
        return 2
    else:
        return 1
    
# Частоту заказов делим на 3 группы, так как за последний год у большей части покупателей было либо 0 либо 1 заказ  

def f_score(x):
    if x < 1:
        return 1
    if x >= 1 and x<=2:
        return 2
    else:
        return 3
    
# Сумму покупок делим на 5 групп в зависимости от количества потраченных денег на основании квантилей

def m_score(x):
    if x <= quintiles['Monetary'][.2]:
        return 1
    elif x <= quintiles['Monetary'][.4]:
        return 2
    elif x <= quintiles['Monetary'][.6]:
        return 3
    elif x <= quintiles['Monetary'][.8]:
        return 4
    else:
        return 5 

In [234]:
#  Добавляем в итоговый датафрейм столбцы с каждой оценкой и столбец общим RFM-рейтингом каждого пользователя
rfm['R'] = rfm['Recency'].apply(lambda x: r_score(x))
rfm['F'] = rfm['Frequency'].apply(lambda x: f_score(x))
rfm['M'] = rfm['Monetary'].apply(lambda x: m_score(x))
rfm['RFM_Score'] = rfm['R'].map(str) + rfm['F'].map(str) + rfm['M'].map(str)

In [235]:
rfm

,customer_unique_id,Recency,Frequency,Monetary,R,F,M,RFM_Score
0,0000366f3b9a7992bf8c76cfdf3221e2,116,1,129.90,4,2,4,424
1,0000b849f77a49e4a4ce2b2a4ca5be3f,119,1,18.90,4,2,2,422
2,0000f46a3911fa3c0805444483337064,542,0,0.00,1,1,1,111
3,0000f6ccb0745a6a4b88665a16c9f078,326,1,25.99,2,2,2,222
4,0004aac84e0df4da2b147fca70cf8255,293,1,180.00,2,2,5,225
...,...,...,...,...,...,...,...,...
95415,fffcf5a5ff07b0908bd4e2dbc735a684,452,0,0.00,1,1,1,111
95416,fffea47cd6d3cc0a88bd621562a9d061,267,1,64.89,3,2,3,323
95417,ffff371b4d645b6ecea244b27531430a,573,0,0.00,1,1,1,111
95418,ffff5962728ec6157033ef9805bacc48,124,1,115.00,4,2,4,424
